In [1]:
import pandas as pd
import numpy as np
import string
import re
from nltk.corpus import stopwords

In [23]:
#Load the file
data_set = pd.read_csv('train.csv')

In [24]:
#----------------------Feature Engineering begin----------------------
#Contractions Map
contraction_map = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"how're": "how are",
"you've": "you have"
}

In [5]:
def expand_contractions(text, contraction_mapping=contraction_map):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [25]:
data_set['No_contractions'] = data_set['text'].apply(lambda x:expand_contractions(x))

In [26]:
data_set.head(5)

,id,keyword,location,text,target,No_contractions
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake M...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to shelter in place are be...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation or..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as ...


In [27]:
#Function to perform the basic text cleaning
def text_cleaner(input_text):
    input_text=input_text.lower()
    input_text = re.sub('^rt', '', input_text)
    input_text = re.sub('\[.*?\]', '', input_text)
    input_text = re.sub(r"http\S+",'',input_text)
    input_text = re.sub('<.*?>+', '', input_text)
    input_text = re.sub('@\w+', '', input_text)
    input_text = re.sub('[%s]' % re.escape(string.punctuation), '', input_text)
    input_text = re.sub('\n', '', input_text)
    input_text = re.sub('\w*\d\w*', '', input_text)
    return input_text

In [28]:
data_set['Cleaned_text'] = data_set['No_contractions'].apply(lambda x:text_cleaner(x))

In [29]:
data_set.head(5)

,id,keyword,location,text,target,No_contractions,Cleaned_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake M...,our deeds are the reason of this earthquake ma...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to shelter in place are be...,all residents asked to shelter in place are be...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation or...",people receive wildfires evacuation orders in...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as ...,just got sent this photo from ruby alaska as s...


In [30]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words= stopwords.words('english')
stop_words.remove('no')
stop_words.remove('not')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diwak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
#Remove stop words
data_set['No_stop_words'] = data_set['Cleaned_text'].str.split() \
    .apply(lambda x: [word for word in x if word not in stop_words]) \
    .apply(lambda x: ' '.join(x))

In [32]:
#Hashtags
data_set['N_hash'] = data_set['text'].str.findall(r'#(\w+)').apply(lambda x : len(x))

In [33]:
#Length of original tweet
data_set['N_tweet'] = data_set['text'].str.len()

In [34]:
#Capital letters in the tweet
def find_caps(input_text):
    n_caps = len(re.findall('([A-Z])', input_text))
    return n_caps
data_set['N_caps'] = data_set['text'].apply(lambda x:find_caps(x))

In [35]:
#Find unique word count
data_set['N_unique_word'] = data_set['Cleaned_text'].str.split().apply(lambda x:len(set(x)))

In [36]:
data_set.head()

,id,keyword,location,text,target,No_contractions,Cleaned_text,No_stop_words,N_hash,N_tweet,N_caps,N_unique_word
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake M...,our deeds are the reason of this earthquake ma...,deeds reason earthquake may allah forgive us,1,69,10,13
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada,forest fire near la ronge sask canada,forest fire near la ronge sask canada,0,38,5,7
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to shelter in place are be...,all residents asked to shelter in place are be...,residents asked shelter place notified officer...,0,133,2,18
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation or...",people receive wildfires evacuation orders in...,people receive wildfires evacuation orders cal...,1,65,1,7
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as ...,just got sent this photo from ruby alaska as s...,got sent photo ruby alaska smoke wildfires pou...,2,88,3,15


In [37]:
#Stem
stemmer = nltk.stem.PorterStemmer()
def stemmer_output(input_string):    
    for token in input_string:
        output_string=stemmer.stem(input_string)
        return output_string

In [38]:
data_set['Stemmed_text'] = data_set['No_stop_words'].apply(lambda x:stemmer_output(x))
#--------------------------Feature engineering stop-------------------------

In [39]:
#--------------------------Building model------------------------------
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [40]:
X_train, X_test, y_train, y_test = train_test_split(data_set['Stemmed_text'], data_set['target'], test_size=0.2, random_state=40)

In [41]:
def cv(data):
    count_vectorizer = CountVectorizer()
    emb = count_vectorizer.fit_transform(data.values.astype('U'))
    return emb, count_vectorizer

X_train_counts, count_vectorizer = cv(X_train)
X_test_counts = count_vectorizer.transform(X_test)

In [42]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)

In [43]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted')             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1

accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.783, precision = 0.783, recall = 0.783, f1 = 0.782


In [44]:
#TF-IDF
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()
    train = tfidf_vectorizer.fit_transform(data.values.astype('U'))
    return train, tfidf_vectorizer

X_train_tfidf, tfidf_vectorizer = tfidf(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [45]:
clf_tfidf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
clf_tfidf.fit(X_train_tfidf, y_train)

y_predicted_tfidf = clf_tfidf.predict(X_test_tfidf)
accuracy_tfidf, precision_tfidf, recall_tfidf, f1_tfidf = get_metrics(y_test, y_predicted_tfidf)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_tfidf, precision_tfidf, 
                                                                       recall_tfidf, f1_tfidf))

accuracy = 0.787, precision = 0.787, recall = 0.787, f1 = 0.786


In [46]:
#Predicting the test set
test_data_set=pd.read_csv('test.csv')
test_data_set.head(5)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [49]:
#Prepare the test data
test_data_set['No_contractions'] = test_data_set['text'].apply(lambda x:expand_contractions(x))

In [50]:
test_data_set['Cleaned_text'] = test_data_set['No_contractions'].apply(lambda x:text_cleaner(x))

In [51]:
test_data_set['No_stop_words'] = test_data_set['Cleaned_text'].str.split() \
    .apply(lambda x: [word for word in x if word not in stop_words]) \
    .apply(lambda x: ' '.join(x))

In [52]:
test_data_set['Stemmed_text'] = test_data_set['No_stop_words'].apply(lambda x:stemmer_output(x))

In [53]:
test_data_set.head(5)

,id,keyword,location,text,No_contractions,Cleaned_text,No_stop_words,Stemmed_text
0,0,NaN,NaN,Just happened a terrible car crash,Just happened a terrible car crash,just happened a terrible car crash,happened terrible car crash,happened terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...","Heard about #earthquake is different cities, s...",heard about earthquake is different cities sta...,heard earthquake different cities stay safe ev...,heard earthquake different cities stay safe ev...
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...","there is a forest fire at spot pond, geese are...",there is a forest fire at spot pond geese are ...,forest fire spot pond geese fleeing across str...,forest fire spot pond geese fleeing across str...
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfires,apocalypse lighting spokane wildfires,apocalypse lighting spokane wildfir
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kills in china and taiwan,typhoon soudelor kills china taiwan,typhoon soudelor kills china taiwan


In [54]:
X_test_dataset = tfidf_vectorizer.transform(test_data_set['Stemmed_text'])

AttributeError: 'NoneType' object has no attribute 'lower'

In [82]:
test_data_set['Stemmed_text'].isnull().sum()

1

In [83]:
test_data_set['Stemmed_text'].fillna("ignore", inplace=True)

In [84]:
X_test_dataset = tfidf_vectorizer.transform(test_data_set['Stemmed_text'])

In [85]:
y_predicted_tfidf = clf_tfidf.predict(X_temp_test_dataset)

In [86]:
print(len(y_predicted_tfidf))

3263


In [91]:
test_data_set['target'] = y_predicted_tfidf

In [92]:
test_data_set.head(5)

,id,keyword,location,text,No_contractions,Cleaned_text,No_stop_words,Stemmed_text,target
0,0,NaN,NaN,Just happened a terrible car crash,Just happened a terrible car crash,just happened a terrible car crash,happened terrible car crash,happened terrible car crash,1
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...","Heard about #earthquake is different cities, s...",heard about earthquake is different cities sta...,heard earthquake different cities stay safe ev...,heard earthquake different cities stay safe ev...,1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...","there is a forest fire at spot pond, geese are...",there is a forest fire at spot pond geese are ...,forest fire spot pond geese fleeing across str...,forest fire spot pond geese fleeing across str...,1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfires,apocalypse lighting spokane wildfires,apocalypse lighting spokane wildfir,1
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kills in china and taiwan,typhoon soudelor kills china taiwan,typhoon soudelor kills china taiwan,1


In [94]:
output_df=test_data_set[['id','target']]

In [95]:
output_df.head(5)

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [96]:
output_df.to_csv('submission.csv')